# Breaking Caesar ciphers
Now we can enciper and decipher with a Caesar cipher, the next step is to write a program to automatically break enciphered messages. 

### Want to know more?
I've written more about [writing programs for ciphers](https://work.njae.me.uk/tag/codes-and-ciphers/) on [my blog](https://work.njae.me.uk/). You can also grab more [code from Github](https://github.com/NeilNjae/cipher-tools/). Feel free to dive in and take a look!

# The basic idea
The basic model is simple: we'll get the computer to try all the keys and see which is the best.

"Trying all the keys" is simple: for a Caesar cipher, we just run through the shifts 0–25.

Finding which is "best" is somewhat tricker. How do we know which is the best possible decryption? For complex ciphers, and long ciphertexts, we don't want to rely on a human to make the decision. 

## Recap: Caesar ciphers
We're just repeating the Caesar cipher implmentation from the previous notebook.

In [1]:
import string

In [2]:
def pos(letter): 
    """Return the position of a letter in the alphabet (0-25)"""
    if letter in string.ascii_lowercase:
        return ord(letter) - ord('a')
    elif letter in string.ascii_uppercase:
        return ord(letter) - ord('A')
    else:
        raise ValueError('pos requires input of {} to be an ascii letter'.format(letter))

In [3]:
def unpos(number): 
    """Return the letter in the given position in the alphabet (mod 26)"""
    return chr(number % 26 + ord('a'))

In [4]:
def caesar_encipher_letter(letter, shift):
    """Encipher a letter, given a shift amount"""
    if letter in string.ascii_letters:
        cipherletter = unpos(pos(letter) + shift)
        if letter in string.ascii_uppercase:
            return cipherletter.upper()
        else:
            return cipherletter
    else:
        return letter      

In [5]:
def caesar_encipher(message, shift):
    """Encipher a message with the Caesar cipher of given shift"""
    enciphered = ""
    for character in message:
        enciphered += caesar_encipher_letter(character, shift)
    return enciphered

In [6]:
def caesar_decipher(message, shift):
    """Decipher a message with the Caesar cipher of given shift"""
    return caesar_encipher(message, -shift)

In [7]:
caesar_encipher('This is a test message.', 4)

'Xlmw mw e xiwx qiwweki.'

In [8]:
caesar_decipher('Xlmw mw e xiwx qiwweki.', 4)

'This is a test message.'

# The New Stuff

## Trying all the keys

Remember, our approach to automatically breaking ciphers is simple:

1. Try all the keys
2. For each key, decrypt the message and "score" it
3. Pick the key with the best "score"

Trying all the keys is easy. Let's try it for a Caesar cipher. Here's an enciphered message:

In [9]:
ciphertext = caesar_encipher('This is a test message.', 4)
ciphertext

'Xlmw mw e xiwx qiwweki.'

And let's try all the keys, seeing what they give.

In [10]:
for key in range(26):
    plaintext = caesar_decipher(ciphertext, key)
    print(plaintext, '<=', key)

Xlmw mw e xiwx qiwweki. <= 0
Wklv lv d whvw phvvdjh. <= 1
Vjku ku c vguv oguucig. <= 2
Uijt jt b uftu nfttbhf. <= 3
This is a test message. <= 4
Sghr hr z sdrs ldrrzfd. <= 5
Rfgq gq y rcqr kcqqyec. <= 6
Qefp fp x qbpq jbppxdb. <= 7
Pdeo eo w paop iaoowca. <= 8
Ocdn dn v ozno hznnvbz. <= 9
Nbcm cm u nymn gymmuay. <= 10
Mabl bl t mxlm fxlltzx. <= 11
Lzak ak s lwkl ewkksyw. <= 12
Kyzj zj r kvjk dvjjrxv. <= 13
Jxyi yi q juij cuiiqwu. <= 14
Iwxh xh p ithi bthhpvt. <= 15
Hvwg wg o hsgh asggous. <= 16
Guvf vf n grfg zrffntr. <= 17
Ftue ue m fqef yqeemsq. <= 18
Estd td l epde xpddlrp. <= 19
Drsc sc k docd wocckqo. <= 20
Cqrb rb j cnbc vnbbjpn. <= 21
Bpqa qa i bmab umaaiom. <= 22
Aopz pz h alza tlzzhnl. <= 23
Znoy oy g zkyz skyygmk. <= 24
Ymnx nx f yjxy rjxxflj. <= 25


From looking at the generated plaintexts, you can probably spot that key 4 gives a sensible text and the others don't. But if you didn't know that `4` was the correct key, it could take you a few moments to find it in the mass of text. It gets harder if the text is longer, and harder still if there are more keys. We want to get a machine to recognise text that looks like English.

# A language model

The idea is to have a _model_ of what English text looks like. When we're presented with a possible plaintext, we can see how well that text fits the model. The better it fits, the more like "ideal English" it is. When we run through all the keys, we pick the key that gives a plaintext that looks most like English.

![Monkey typing](Monkey-typing.jpg) 

My approach takes the idea from the apocryphal story that an infinite number of moneys, with an infinite number of typewriters, will eventually create the complete works of Shakespeare. As the computer tries each key, generating a possible plaintext, it can score the possible plaintext by how likely it would be for a monkey, completely randomly, to generate that plaintext. 

That idea isn't that helpful when all the letters are equally likely. But if the monkey is using a keyboard where the keys are sized in proportion to how often they appear in English, we have something. The diagram below gives an idea of what such a keyboard could look like. A monkey using that keyboard, htting keys at random, will produce something like *treattlpis* more often than *nziuechjtk*.

![English letters by proportion](letter-treemap.png) 

That allows us to score how close a piece of text is to English. If we can work out the probability htat a random monkey would produce our possible plaintext, we can choose the key which produces the most likely plaintext.

(This model is also called a _bag of letters_ model, as it's the same as taking all the letters in the text and putting them in a bag, losing all idea of the order of letters. While we lose a lot of information with this, it's actually good enough for our purposes.)

## Finding letter probabilities
How do we find these probabilities of the letters of English? The easy answer is to simply read a lot of text, counting the letters. Let's use three large texts: the [complete works of Shakespeare](https://www.gutenberg.org/ebooks/100), [War and Peace](https://www.gutenberg.org/ebooks/2600), and [The Adventures of Sherlock Holmes](https://www.gutenberg.org/ebooks/48320), all from [Project Gutenberg](https://www.gutenberg.org/wiki/Main_Page).

The Python [`collections.Counter()`](https://docs.python.org/3/library/collections.html#collections.Counter) object from the standard library does a good job of counting letters for us. If we pass a `Counter` a sequence of characters, it will count them for us:

In [11]:
import collections
collections.Counter('this is some test text')

Counter({'t': 5,
         'h': 1,
         'i': 2,
         's': 4,
         ' ': 4,
         'o': 1,
         'm': 1,
         'e': 3,
         'x': 1})

If we `update` the `Counter`, it will update the counts for us:

In [12]:
counts = collections.Counter()
counts.update('this is some text')
counts

Counter({'t': 3,
         'h': 1,
         'i': 2,
         's': 3,
         ' ': 3,
         'o': 1,
         'm': 1,
         'e': 2,
         'x': 1})

In [13]:
counts.update('here is some more text')
counts

Counter({'t': 5,
         'h': 2,
         'i': 3,
         's': 5,
         ' ': 7,
         'o': 3,
         'm': 3,
         'e': 7,
         'x': 2,
         'r': 2})

This allows us to easily combine the counts of letters in all the texts, then write them to a file. 

In [14]:
corpora = ['shakespeare.txt', 'sherlock-holmes.txt', 'war-and-peace.txt']
counts = collections.Counter()

# count all the characters
for corpus in corpora:
    text = open(corpus, 'r').read().lower()
    counts.update(text)

    
# keep just the letters    
letter_counts = {}
for l in counts:
    if l in string.ascii_lowercase:
        letter_counts[l] = counts[l]
        
# sort the letters, most common first
sorted_letters = sorted(letter_counts, key=counts.get, reverse=True)

# write the counts to a file, in case we need them later
with open('count_1l.txt', 'w') as f:
    for l in sorted_letters:
        f.write("{0}\t{1}\n".format(l, letter_counts[l]))
        print("{0}\t{1}".format(l, letter_counts[l]))

e	758091
t	560576
o	504520
a	490124
i	421240
n	419374
h	416369
s	404473
r	373599
d	267917
l	259023
u	190269
m	172199
w	154157
y	143040
c	141094
f	135318
g	117888
p	100690
b	92919
v	65297
k	54248
x	7414
j	6679
q	5499
z	3577


For interest, how many letters were counted?

In [15]:
sum(letter_counts.values())

6265594

At five letters per word on average, how many words is that? (Pretty printing to show the commas as separators in the number.)

In [16]:
"{:,}".format(sum(letter_counts.values()) / 5)

'1,253,118.8'

## Using the language model
Now we have the letter counts, we need to use them to score possible plaintexts. There are two stages here:
1. Convert the counts into probabilities;
2. Score a piece of text for how probable it is (by the random monkey metric).

## Converting the counts to probabilities
This just requires working out what proportion each letter is of the total counts. 

The probability of a letter occuring by chance is just the number of times that letter appears, divided by the total number of letters. 

Whe have the number of each letter in `letter_counts`: `letter_counts[letter]` is how often a letter appeared in the corpus. 

We can find the total of all the counts with `sum(letter_counts.values())`.

Division in Python is done with the `/` operator, like this:

In [17]:
12 / 4

3.0

In [18]:
12 / 5

2.4

## Your turn: find `letter_probability`s

Complete the code below. If you're stuck, you can [see the solution](letter_probability-solution.ipynb).

In [26]:
letter_probability = {}
count_sum = # Write your code here
for letter in letter_counts:
    letter_probability[letter] = # Write your code here

letter_probability

SyntaxError: invalid syntax (<ipython-input-26-d820b0f07ba3>, line 2)

## Score a piece of text
We can use these to find the probability of a sequence of letters by finding the probability of each letter, then multiplying them all together. What we end up with is the probability of this piece of text being typed out by the random money I introduced at the top of this notebook.

Note that the `letter_probability` dict only holds lower-case letters, so we make sure to convert the text to lowercase before tying to score it. 

In [20]:
def text_probability(text):
    prob = 1.0
    for letter in text.lower():
        if letter in letter_probability:
            prob *= letter_probability[letter]
    return prob

In [21]:
text_probability('hello')

1.1064798867415649e-06

In [22]:
text_probability('this is a longer piece of text, like what we might find in a message')

1.3917178596173123e-68

That's quite a small probability. To see how small, let's put it in a more human-readable format:

In [23]:
'{:0.70f}'.format(text_probability('this is a longer piece of text, like what we might find in a message'))

'0.0000000000000000000000000000000000000000000000000000000000000000000139'

The final values will be very small even longer texts. There's a danger that with a long text, we'll end up with a number that's too small to represent. 

We can get around this by using the "trick" of taking logarithms of probabilities. As numbers get smaller, their logarithms get smaller, but much less quickly. This means we can still handle the probability of long texts, while still being able to see which is the most probable. 

To find the logarithm of a number, we use the `math.log10()` function.

In [24]:
import math

math.log10(1000)

3.0

In [25]:
math.log10(1)

0.0

In [26]:
math.log10(0.1)

-1.0

In [27]:
math.log10(0.005)

-2.3010299956639813

When we multiply numbers, we _add_ their logarithms. Here's a multiplication:

In [28]:
0.002 * 5

0.01

Here's the logarithm of the result.

In [29]:
math.log10(0.002 * 5)

-2.0

Here are the logarithms of the two numbers.

In [30]:
math.log10(0.002), math.log10(5)

(-2.6989700043360187, 0.6989700043360189)

To multiply them together, we add their logarithms, and get the same answer as above.

In [31]:
math.log10(0.002) + math.log10(5)

-2.0

(If you're interested, we can get the original number back from the logarithm by raising 10 to a power:)

In [32]:
10 ** -2.0

0.01

Before, we multiplied the probabilities together to get the probability of the sentence. With logarithms, to get the logarithm of the probability, we add all the log probabilities of the letters. As we're only interested in which is the _most_ likely, we don't need to convert _from_ logarithms anywhere: the most likely plaintext is just the one with the highest probability, which is the highest log probability. 

## Your turn: `log_letter_probability`
Knowing what you know from above, fill the `log_letter_probability` dict with the log probabilities of the letters. 

Follow the code for filling in `letter_probability`, but take the logarithm of the fraction of letters.

If you get stuck, you can [see the solution](log_letter_probability-solution.ipynb).

In [77]:
log_letter_probability = {}

# Write your code here

log_letter_probability

{}

## Your turn: `log_text_probability()`
Using the `log_letter_probability` dict, write the `log_text_probability()` function to give the log probability of a piece of text. 

You should follow the code from `text_probability()` above, but start with a `log_probability` of zero, and _add_ the log probabilities of the letters.

If you get stuck, you can [see the solution](log_text_probability-solution.ipynb).

In [78]:
def log_text_probability(letters):
    # Write your code here

We can use that to find some log probabilities:

In [35]:
log_text_probability('hello')

-5.956056476139383

In [36]:
log_text_probability('hello there')

-11.240906451276237

In [37]:
log_text_probability('this is a longer piece of text, like what we might find in a message')

-67.8564487995177

Much better values!

We've ended up with a simple _model_ of the English language, which we can use to judge how likely a given piece of text is actually English. 

## Breaking Caesar ciphers
Finally, we can use `log_text_probability()` to score possible plaintexts, and hence automatically break Caesar ciphers!

Back at the top of the notebook, we showed how to try all keys and generate the corresponding plaintext for each:

In [38]:
ciphertext = caesar_encipher('This is a test message.', 4)

for key in range(26):
    plaintext = caesar_decipher(ciphertext, key)
    print(plaintext, '<=', key)

Xlmw mw e xiwx qiwweki. <= 0
Wklv lv d whvw phvvdjh. <= 1
Vjku ku c vguv oguucig. <= 2
Uijt jt b uftu nfttbhf. <= 3
This is a test message. <= 4
Sghr hr z sdrs ldrrzfd. <= 5
Rfgq gq y rcqr kcqqyec. <= 6
Qefp fp x qbpq jbppxdb. <= 7
Pdeo eo w paop iaoowca. <= 8
Ocdn dn v ozno hznnvbz. <= 9
Nbcm cm u nymn gymmuay. <= 10
Mabl bl t mxlm fxlltzx. <= 11
Lzak ak s lwkl ewkksyw. <= 12
Kyzj zj r kvjk dvjjrxv. <= 13
Jxyi yi q juij cuiiqwu. <= 14
Iwxh xh p ithi bthhpvt. <= 15
Hvwg wg o hsgh asggous. <= 16
Guvf vf n grfg zrffntr. <= 17
Ftue ue m fqef yqeemsq. <= 18
Estd td l epde xpddlrp. <= 19
Drsc sc k docd wocckqo. <= 20
Cqrb rb j cnbc vnbbjpn. <= 21
Bpqa qa i bmab umaaiom. <= 22
Aopz pz h alza tlzzhnl. <= 23
Znoy oy g zkyz skyygmk. <= 24
Ymnx nx f yjxy rjxxflj. <= 25


## Your turn: show the log probability scores
Generate the same table as above, but this time give the log probability score for each generated plaintext.

If you get stuck, you can [see the solution](show-log-probs-solution.ipynb).

In [66]:
# Write your code here

If you do it, you can see that all the possible plaintexts score -25 or lower, apart from one. And that's the correct key.

The next step is to keep track of the _best_ key. 

We work through all 26 keys, deciphering the text with each one. For each possible plaintext, we use the `log_text_probability()` function to score how good that plaintext is. As we go through, we keep track of the best probability and the key which generated it. 

At the end, we return the best key and probability.

We start by initialising the `best_key` and `best_prob` to extreme values which will be instantly overridden (`float('-inf')` is negative infinity, a _really_ bad score):

In [68]:
best_key = 0
best_prob = float('-inf')

We can use this code to update the best key and best probability:

In [ ]:
# Don't run this cell: it's just a code fragment and will give an error
if log_prob_score > best_prob:
    best_prob = log_prob_score
    best_key = key

## Your turn: find the best key
In the code you wrote above, you showed the score for each candidate plaintext. 

Extend that to keep track of the best score over all the keys. Copy you code below and add in the couple of fragments above for keeping track of the best key.

If you get stuck, you can [see the solution](show-best-log-prob.ipynb).


In [ ]:
# Write your code here

## Your turn: wrap it in a function

Let's make this codebreaking snippet more usable by wrapping it in a function definition. 

Fill in the definition of `caesar_break()` using the code you've just written.

If you're stuck, you can [see the solution](caesar_break-solution.ipynb).

In [70]:
def caesar_break(message):
    """Breaks a Caesar cipher using frequency analysis"""

    # Write your code here
    
    # Return the best we found
    return best_key, best_prob

Let's see if it works. We encipher a message, then see if `caesar_break` returns the correct key.

In [43]:
key, score = caesar_break(ciphertext)
key, score

(4, -20.86915553487099)

Success!

Let's try some other messages.

In [44]:
ct = caesar_encipher("""Here's another test message to see if the caesar 
breaking code works well.""", 17)

key, score = caesar_break(ct)
key, score

(17, -73.03537110834638)

Let's try it on something larger: the ciphertext from the [National Cipher Challenge](https://www.cipherchallenge.org/) [2016 challenge 1](https://2016.cipherchallenge.org/challenges/challenge-1/).

In [45]:
c1a = open('2016-1a.ciphertext').read()
print(c1a)

PIZZG,
Q PIDM AKIVVML BPM MVKZGXBML VWBM BPM XWTQKM NWCVL WV RIUMTQI'A LMAS IVL IBBIKPML QB NWZ GWC BW TWWS IB. BPM XWTQKM LMKZGXBML QB NWZ BPMUAMTDMA (QB QA DMZG ABZIQOPBNWZEIZL WVKM GWC ZMITQAM BPIB QB PIA JMMV EZQBBMV JIKSEIZLA - QB RCAB CAMA I KIMAIZ APQNB KQXPMZ). BPM WNNQKMZ QV KPIZOM WN BPM QVDMABQOIBQWV UILM QB KTMIZ BW UM BPIB PM BPQVSA BPQA XZWDMA RIUMTQI'A LMIBP QA "RCAB" I XMZAWVIT BZIOMLG. KIZMTMAA CAM WN BPM EWZL "RCAB" MDMV QN PM QA ZQOPB, JCB Q LWV'B BPQVS PM QA. Q PIDM AXWSMV BW PMZ KWTTMIOCMA, IVL RIUMTQI LWMAV'B ABZQSM UM IA I RCUXMZ. APM EIA XZMBBG LZQDMV IVL PMZ EWZS EIA OWQVO MFBZMUMTG EMTT. IXXIZMVBTG APM EIA CVPIXXG IJWCB PMZ JWGNZQMVL TMIDQVO, JCB I YCQKS AKIV WN PMZ AMIZKP PQABWZG ACOOMABA APM EIA XZMBBG IKBQDM QV BZGQVO BW BZIKS PQU LWEV. BPM XWTQKM BPQVS BPIB APWEA PWE LMAXMZIBM APM EIA. Q BPQVS QB APWEA BPIB APM EIAV'B BPM AWZB BW OQDM CX MIAQTG.
WV WVM BPQVO Q LW IOZMM EQBP BPM XWTQKM, QB LWMAV'B AMMU DMZG TQSMTG BPIB I XPGAQKQAB EWZSQVO WV OZIDQBG EIDMA Q

In [46]:
key, score = caesar_break(c1a)
key, score

(8, -1547.4874599084396)

In [47]:
print(caesar_decipher(c1a, key))

HARRY,
I HAVE SCANNED THE ENCRYPTED NOTE THE POLICE FOUND ON JAMELIA'S DESK AND ATTACHED IT FOR YOU TO LOOK AT. THE POLICE DECRYPTED IT FOR THEMSELVES (IT IS VERY STRAIGHTFORWARD ONCE YOU REALISE THAT IT HAS BEEN WRITTEN BACKWARDS - IT JUST USES A CAESAR SHIFT CIPHER). THE OFFICER IN CHARGE OF THE INVESTIGATION MADE IT CLEAR TO ME THAT HE THINKS THIS PROVES JAMELIA'S DEATH IS "JUST" A PERSONAL TRAGEDY. CARELESS USE OF THE WORD "JUST" EVEN IF HE IS RIGHT, BUT I DON'T THINK HE IS. I HAVE SPOKEN TO HER COLLEAGUES, AND JAMELIA DOESN'T STRIKE ME AS A JUMPER. SHE WAS PRETTY DRIVEN AND HER WORK WAS GOING EXTREMELY WELL. APPARENTLY SHE WAS UNHAPPY ABOUT HER BOYFRIEND LEAVING, BUT A QUICK SCAN OF HER SEARCH HISTORY SUGGESTS SHE WAS PRETTY ACTIVE IN TRYING TO TRACK HIM DOWN. THE POLICE THINK THAT SHOWS HOW DESPERATE SHE WAS. I THINK IT SHOWS THAT SHE WASN'T THE SORT TO GIVE UP EASILY.
ON ONE THING I DO AGREE WITH THE POLICE, IT DOESN'T SEEM VERY LIKELY THAT A PHYSICIST WORKING ON GRAVITY WAVES I

# Over to you
If you (or a friend) generated some Caesar-enciphered messages before, can `caesar_break()` crack the code and recover the message?